In [1]:
%reload_ext autoreload
%autoreload 2
%cd llama3/llama

/home/tbai4/llama3/llama


/home/tbai4/llama3/.venv/lib/python3.12/site-packages/IPython/core/magics/osm.py:417: UserWarning: This is now an optional IPython functionality, setting dhist requires you to install the `pickleshare` library.
  self.shell.db['dhist'] = compress_dhist(dhist)[-100:]


In [2]:
import os
from llama import Workflow, Llama

os.environ["RANK"] = "0"
os.environ["WORLD_SIZE"] = "1"
os.environ["MASTER_ADDR"] = "localhost"
os.environ["MASTER_PORT"] = "29500"

workflow = Workflow.build(
    ckpt_dir='/scratch4/jeisner1/tjbai/llama_8b',
    tokenizer_path='/scratch4/jeisner1/tjbai/llama_8b/tokenizer.model',
    max_seq_len=4096,
    max_batch_size=8,
    model_parallel_size=1,
    max_nodes=100,
)

llama = Llama(workflow.model, workflow.tokenizer)

> initializing model parallel with size 1
> initializing ddp with size 1
> initializing pipeline with size 1
Loaded in 13.38 seconds


In [20]:
import os
import torch
import torch.nn.functional as F

os.environ['TORCH_SHOW_CPP_STACKTRACES'] = '1'

def check_flash_attention(q, k, v):
    try:
        with torch.backends.cuda.sdp_kernel(enable_flash=True, enable_math=False, enable_mem_efficient=False):
            out = F.scaled_dot_product_attention(q, k, v)    
        print("Backend selection succeeded")
        return True
    except RuntimeError as e:
        print(f"Error: {str(e)}")
        if "Flash attention" in str(e):
            print("Flash Attention attempted but failed - likely not supported for this input")
        return False

B, H, L, D = 1, 8, 1024, 64
q = torch.randn(B, H, L, D, device='cuda', dtype=torch.bfloat16)
k = torch.randn(B, H, L, D, device='cuda', dtype=torch.bfloat16)
v = torch.randn(B, H, L, D, device='cuda', dtype=torch.bfloat16)

check_flash_attention(q, k, v)

Backend selection succeeded


True

In [ ]:
import time

workflow.model.reshape_cache(32)

def test_reencoding():
    device = torch.device("cuda" if torch.cuda.is_available() else "cpu")
    model = workflow.model
    tokenizer = workflow.tokenizer

    batch_sizes = [4, 8, 16, 32]
    seq_lengths = [128, 256, 512, 1024]
    num_repeats = 10
    warmup_steps = 3

    print(f"Benchmarking on {device}\n")
    print(f"{'Batch Size':<10} | {'Seq Length':<10} | {'Time (ms)':<10}")

    for batch_size in batch_sizes:
        for seq_len in seq_lengths:
            input_ids = torch.full((batch_size, seq_len), tokenizer.eot_id, device=device)
            
            with torch.no_grad():
                for _ in range(warmup_steps):
                    _ = model.forward(input_ids, start_pos=0)

            total_time = 0.0
            with torch.no_grad():
                for _ in range(num_repeats):
                    torch.cuda.synchronize()
                    start_time = time.time()
                    _ = model(input_ids, start_pos=0)
                    torch.cuda.synchronize()
                    total_time += (time.time() - start_time) * 1000

            avg_time = total_time / num_repeats
            print(f"{batch_size:<10} | {seq_len:<10} | {avg_time:.2f}")

test_reencoding()

In [4]:
import json
from pathlib import Path

def load_math_problems(root_dir, split, problem_types):
    problems = []
    root = Path(root_dir) / split
    
    for problem_type in problem_types:
        type_dir = root / problem_type
        if not type_dir.exists():
            continue
            
        for prob_file in type_dir.glob("*.json"):
            with open(prob_file) as f:
                problem = json.load(f)
                problems.append(problem)
    
    return problems

problems = load_math_problems('../../MATH', 'train', ['counting_and_probability'])

In [8]:
from llama.benchmark.tot import benchmark_tot

baseline_result, cached_result = benchmark_tot(
    llama=llama,
    workflow=workflow,
    problem=problems[0]['problem'],
    branching_factor=8,
    voters=4,
)

In [10]:
print(baseline_result['mean'])
print(cached_result['mean'])

20.7331447271863
20.59817117068451


In [52]:
output = baseline_result['outputs'][0]

for proposal in output['proposal_content']:
    print(proposal)
    print('###')
    
for vote in output['vote_content']:
    print(vote)
    print('###')
    
print(output['final_content'])

APPROACH:

1.  Consider the total number of possible groups of three knights chosen from the 25 seated at the round table, and then count the number of groups where at least two knights are sitting next to each other.
2.  To count the groups where at least two knights are sitting next to each other, consider the complementary scenario where no two knights are sitting next to each other, and use this to calculate the desired probability.
###
APPROACH:

1.  Calculate the total number of ways to choose 3 knights from 25, which is $\binom{25}{3}$.
2.  Use complementary counting to find the number of ways to choose 3 knights such that none of them are sitting next to each other, and then subtract this from the total number of ways to choose 3 knights.
###
APPROACH:

1. First, consider the total number of ways to choose 3 knights from 25, and the total number of ways they can be seated around the round table.
2. Next, determine the number of ways to choose 3 knights such that at least two of

In [51]:
output = cached_result['outputs'][0]

for proposal in output['proposal_tokens']:
    print(workflow.tokenizer.decode(proposal))
    print('###')
    
for vote in output['vote_tokens']:
    print(workflow.tokenizer.decode(vote))
    print('###')
    
print(workflow.tokenizer.decode(output['final_tokens']))

Solution #1:

APPROACH:

1.  Consider the total number of possible groups of three knights chosen from the 25 seated at the round table, and then count the number of groups where at least two knights are sitting next to each other.
2.  To count the groups where at least two knights are sitting next to each other, consider the complementary scenario where no two knights are sitting next to each other, and use this to calculate the desired probability.
###
Solution #2:

APPROACH:

1.  Calculate the total number of ways to choose 3 knights from 25, which is $\binom{25}{3}$.
2.  Use complementary counting to find the number of ways to choose 3 knights such that none of them are sitting next to each other, and then subtract this from the total number of ways to choose 3 knights.
###
Solution #3:

APPROACH:

1. First, consider the total number of ways to choose 3 knights from 25, and the total number of ways they can be seated around the round table.
2. Next, determine the number of ways to 

In [27]:
llama.model.reshape_cache(new_batch_size=4)
results_baseline = benchmark(
    fn=partial(tot_baseline, llama=llama),
    args=cases,
)
print([r['mean'] for r in results_baseline])

[np.float64(57.00386735699139), np.float64(52.51334427534603), np.float64(41.15609271898866), np.float64(52.84291560975835), np.float64(32.12614595838822)]


In [7]:
workflow.reset()

[generate_prompt, evaluate_prompt] = workflow.insert([
    {
        'messages': [
            {'role': 'system', 'content': 'You will be shown a math problem. Provide a high-level summary of how to solve the problem.'},
            {'role': 'user', 'content': 'Here is the problem: How many ways can you sit 10 different people around a round table?'}
        ],
        'parent_ids': [],
    },
    {
        'messages': [
            {'role': 'system', 'content': 'Given a math problem and numerous proposed solutions, echo back to the user all of the solutions.'},
        ],
        'parent_ids': []
    }
])

resp1, resp2 = workflow.insert([
    {
        'messages': [{'role': 'assistant', 'content': 'We can possibly use binomial coefficients to solve this problem via combinations.'}],
        'parent_ids': [generate_prompt['id']],
    },
    {
        'messages': [{'role': 'assistant', 'content': 'We can consider all 10 permutations and cancel out the rotational symmetry.'}],
        'parent_ids': [evaluate_prompt['id']],
    },
])

tokens, cached = workflow.step(
    tasks=[
        {
            'header': ('assistant', None),
            'prefill': 'The user proposed doing',
            'parent_ids': [evaluate_prompt['id'], resp2['id'], resp1['id']],

        }
        for _ in range(4)
    ],
    compact=True,
    max_gen_len=128,
    temperature=1.0,
    top_p=1.0,
    seed=1,
)

for i, output in enumerate(tokens):
    print(f'example #{i+1}')
    print(workflow.tokenizer.decode(output))
    print()

/home/tbai4/llama3/llama/workflow.py:122: UserWarning: Multi-node compaction not fully implemented. Use caution.
  warnings.warn("Multi-node compaction not fully implemented. Use caution.")


example #1
The user proposed doing it by dividing, doing long division, and by finding the prime factorization.

example #2
The user proposed doing the problem through all 10 combinations by hand.

 presets later,,  
 Broom being hydraulic daughter Engine  
 roots philosophy independent Kick  
 rag complete resistance dist anesthesia  
 strap orchestra Piper repeating Corner  
 sticky Boy handling appraisal shelter  
 almond substrate Building Tran actual lifestyles rect stage proced cite deserves pay cancelling breakout sanctuary  
 rst stimulate meteor mixture replace observation secular uno saturated science follower campaigns column grease difficult ventures cater assessments unusually slave laws insulated takeover skyline song admit marching Conditional Gala refresh receiving helpless cancellation contains rake sustainable body Hend forearm '-' wherever revolves furnishings=t057 mango transition coil technically prism session pros reflected dissip diss till received acne crossing 

In [8]:
import torch

workflow.model = workflow.model.to(torch.float16)
llama.model = llama.model.to(torch.float16)

completion = llama.chat_completion([[
    {'role': 'system', 'content': 'Answer the user\'s question please.'},
    {'role': 'user', 'content': 'What is the capital of France?'},
    {'role': 'user', 'content': 'What is the capital of Germany?'},
]])

print(completion)

regular_cache_v = llama.model.layers[0].attention.cache_v[0].clone()
regular_cache_k = llama.model.layers[0].attention.cache_k[0].clone()

workflow.reset()

[system] = workflow.insert([
    {
        'messages': [{'role': 'system', 'content': 'Answer the user\'s question please.'}],
        'parent_ids': [],
    },
])

[user_1] = workflow.insert([
    {
        'messages': [{'role': 'user', 'content': 'What is the capital of France?'}],
        'parent_ids': [system['id']],
    },
])

[user_2] = workflow.insert([
   {
        'messages': [{'role': 'user', 'content': 'What is the capital of Germany?'}],
        'parent_ids': [system['id'], user_1['id']],
    }
])

workflow_cache_v = workflow.model.layers[0].attention.cache_v[0].clone()
workflow_cache_k = workflow.model.layers[0].attention.cache_k[0].clone()

for n in range(workflow.cache_len):
    print(n, (workflow_cache_v[n] - regular_cache_v[n]).max())
    print(n, (workflow_cache_k[n] - regular_cache_k[n]).max())
    
workflow.model = workflow.model.bfloat16()
llama.model = llama.model.bfloat16()

[{'generation': {'role': 'assistant', 'content': 'You asked two questions!\n\nThe answers are:\n\n1. The capital of France is Paris.\n2. The capital of Germany is Berlin.'}, 'tokens': [2675, 4691, 1403, 4860, 2268, 791, 11503, 527, 1473, 16, 13, 578, 6864, 315, 9822, 374, 12366, 627, 17, 13, 578, 6864, 315, 10057, 374, 20437, 13]}]
0 tensor(0., device='cuda:0', dtype=torch.float16)
0 tensor(0., device='cuda:0', dtype=torch.float16)
1 tensor(9.5367e-07, device='cuda:0', dtype=torch.float16)
1 tensor(0., device='cuda:0', dtype=torch.float16)
2 tensor(7.6294e-06, device='cuda:0', dtype=torch.float16)
2 tensor(0., device='cuda:0', dtype=torch.float16)
3 tensor(0., device='cuda:0', dtype=torch.float16)
3 tensor(1.5259e-05, device='cuda:0', dtype=torch.float16)
4 tensor(3.8147e-06, device='cuda:0', dtype=torch.float16)
4 tensor(0., device='cuda:0', dtype=torch.float16)
5 tensor(0., device='cuda:0', dtype=torch.float16)
5 tensor(0., device='cuda:0', dtype=torch.float16)
6 tensor(3.8147e-06, d

In [10]:
# load data

import json
from pathlib import Path

def load_math_problems(root_dir, split, problem_types):
    problems = []
    root = Path(root_dir) / split
    
    for problem_type in problem_types:
        type_dir = root / problem_type
        if not type_dir.exists():
            continue
            
        for prob_file in type_dir.glob("*.json"):
            with open(prob_file) as f:
                problem = json.load(f)
                problems.append(problem)
    
    return problems

problems = load_math_problems('../../MATH', 'train', ['counting_and_probability'])

In [28]:
import re
from collections import Counter

cot_prompt = '''
You are a creative problem solver with deep expertise in competition mathematics.
Your goal is to propose a clean and insightful approach to solving the provided problem.
Keep your proposal high-level and CONCISE. Do not attempt to solve the entire problem.

You must format your response as:
APPROACH:
1. (high-level first step)
2. (high-level second step)
'''

trick_prompt = '''
You are a trickster. When the user gives you a math problem, you will give a humorous and terrible solution.
Keep your response concise. Aim for sarcasm and humor.
'''

finish_prompt = '''
You are a creative problem solver with deep expertise in competition mathematics.
You will be provided a problem and a proposed approach to solving it. Using the steps provided, finish solving the problem.
Keep your solution CONCISE and lean on ideas developed in the proposal.
You MUST provide the final answer to the problem and will be graded on correctness.

You must format your response as:
ANSWER: (2-3 sentence summary of solution and final answer)
'''

def format_vote_prompt(n):
    return f'''
    You are a rigorous mathematical evaluator with deep expertise in competition mathematics.
    You will be shown several different solution strategies for a math problem.
    Vote on the best proposal and give a CONCISE justification of your choice.
    You will see {n} proposals, so respond with the proposal 1 through {n}.
    Do not attempt to solve the problem. You only need to evaluate each proposal and select the best option.

    You must format your response as:
    BEST CHOICE: (index of best solution)
    RATIONALE: (1 sentence justification)
    '''

def format_problem(problem):
    return f'Here is the provided problem:\n{problem}'

def parse_choice(text):
    match = re.search(r'BEST CHOICE:\s*(\d+)', text)
    if match:
        return int(match.group(1))
    return None

def solve(problem, branching_factor, voters):
    workflow.reset()

    cot, vote, finish = workflow.insert([
        {
            'messages': [{'role': 'system', 'content': cot_prompt}, {'role': 'user', 'content': format_problem(problem)}],
            'parent_ids': []
        },
        {
            'messages': [{'role': 'system', 'content': format_vote_prompt(branching_factor)}, {'role': 'user', 'content': format_problem(problem)}],
            'parent_ids': []
        },
        {
            'messages': [{'role': 'system', 'content': finish_prompt}, {'role': 'user', 'content': format_problem(problem)}],
            'parent_ids': []
        },
    ])
    
    proposal_tokens, proposal_nodes = workflow.step(
        [
            {
                'header': ('assistant', None),
                'prefill': f'Solution #{i+1}:\n\n',
                'parent_ids': [cot['id']]
            }
            for i in range(branching_factor)
        ],
        compact=False,
        max_gen_len=512,
        temperature=0.7,
        top_p=0.9,
        seed=42,
        debug=False,
    )
    
    vote_tokens, vote_nodes = workflow.step(
        [
            {
                'header': ('assistant', None),
                'prefill': 'BEST CHOICE: ',
                'parent_ids': [vote['id']] + list([p['id'] for p in proposal_nodes])
            }
            for _ in range(voters)
        ],
        compact=False,
        max_gen_len=256,
        temperature=0.7,
        top_p=0.9,
        seed=42,
        debug=False
    )
    
    res = None
    votes = [
        choice for resp in vote_tokens if 
        (choice := parse_choice(workflow.tokenizer.decode(resp))) is not None
    ]
    
    if len(votes) > 0:
        best = Counter(votes).most_common(1)[0][0]
        [res], _ = workflow.step(
            [
                {
                    'header': ('assistant', None),
                    'parent_ids': [finish['id']] + [proposal_nodes[best-1]['id']]
                }
            ],
            compact=False,
            max_gen_len=256,
            temperature=0.7,
            top_p=0.9,
            seed=42,
            debug=False
        )

    return proposal_tokens, vote_tokens, res, votes

import time
s = time.time()
proposal_tokens, vote_tokens, res, votes = solve(problems[0]['problem'], branching_factor=5, voters=10)
print(time.time() - s)

14.485045671463013


In [29]:
for proposal in proposal_tokens:
    print(workflow.tokenizer.decode(proposal))
    print('###')

Solution #1:

APPROACH:
1. First, consider the total number of ways to choose 3 knights out of 25 and find the total number of outcomes.
2. Next, consider the cases where two knights are sitting next to each other and calculate the number of outcomes in each case, then find the total number of outcomes where at least two knights are sitting next to each other.
###
Solution #2:

APPROACH:

1. **Count the total number of ways to choose 3 knights out of 25**: This will involve using combinations, specifically $\binom{25}{3}$.
2. **Count the number of ways to choose 3 knights where at least two are sitting next to each other**: This can be broken down into cases such as two adjacent knights and the third knight being one seat away, or all three knights being adjacent.
###
Solution #3:

APPROACH:

1. Count the total number of ways to choose 3 knights from 25, which is $\binom{25}{3}$.
2. Count the number of ways to choose 3 knights such that at least two of them are sitting next to each oth

In [30]:
for vote in vote_tokens:
    print(workflow.tokenizer.decode(vote))
    print('###')

BEST CHOICE: 1
RATIONALE: This approach is the most direct and straightforward method to solve the problem, allowing for a clear and concise calculation of the probability.
###
BEST CHOICE: 1
RATIONALE: This approach is the most straightforward and direct method to solve the problem, allowing for a clear and systematic count of the desired outcomes.
###
BEST CHOICE: 2
RATIONALE: This approach allows for a clear and systematic way to count the number of favorable outcomes, making it more likely to arrive at the correct solution.
###
BEST CHOICE: 1
RATIONALE: This approach is the most straightforward way to count the number of ways to choose 3 knights such that at least two of them are sitting next to each other, which is the main task of the problem.
###
BEST CHOICE: 2
RATIONALE: This approach allows for a clear and systematic way to count the number of ways to choose 3 knights such that at least two of them are sitting next to each other, making it easier to calculate the probability P

In [14]:
import time

branching_factor = 2
voters = 2

problem = problems[0]['problem']

workflow.reset()

cot, vote, finish = workflow.insert([
    {
        'messages': [{'role': 'system', 'content': cot_prompt}, {'role': 'user', 'content': format_problem(problem)}],
        'parent_ids': []
    },
    {
        'messages': [{'role': 'system', 'content': format_vote_prompt(branching_factor)}, {'role': 'user', 'content': format_problem(problem)}],
        'parent_ids': []
    },
    {
        'messages': [{'role': 'system', 'content': finish_prompt}, {'role': 'user', 'content': format_problem(problem)}],
        'parent_ids': []
    },
])

proposal_tokens, proposal_nodes = workflow.step(
    [
        {
            'header': ('assistant', None),
            'prefill': f'Solution #{i+1}:\n\n',
            'parent_ids': [cot['id']]
        }
        for i in range(branching_factor)
    ],
    compact=False,
    max_gen_len=512,
    temperature=0.7,
    top_p=0.9,
    seed=42,
    debug=False,
)

vote_tokens, vote_nodes = workflow.step(
    [
        {
            'header': ('assistant', None),
            'prefill': 'BEST CHOICE: ',
            'parent_ids': [vote['id']] + list([p['id'] for p in proposal_nodes])
        }
        for _ in range(voters)
    ],
    compact=False,
    max_gen_len=256,
    temperature=0.7,
    top_p=0.9,
    seed=42,
    debug=False
)

In [15]:
for proposal in proposal_tokens:
    print(workflow.tokenizer.decode(proposal))
    print('###')

Solution #1:

APPROACH:
1. First, consider the total number of ways to choose 3 knights out of 25 and find the total number of outcomes.
2. Next, consider the cases where two knights are sitting next to each other and find the number of outcomes in each case. Then, apply the Pigeonhole Principle to find the number of outcomes where at least two knights are sitting next to each other.
###
Solution #2:

APPROACH:

1. **Count the total number of ways to choose 3 knights out of 25**: This will involve using combinations to find the total number of outcomes.
2. **Count the number of ways to choose 3 knights where at least two of them are sitting next to each other**: We can break this down into cases, such as choosing two knights who are next to each other and then choosing the third knight from the remaining knights, or choosing three knights who are all next to each other.
###


In [16]:
for vote in vote_tokens:
    print(workflow.tokenizer.decode(vote))
    print('###')

BEST CHOICE: 2
RATIONALE: This approach allows for a more systematic and detailed count of the number of outcomes where at least two knights are sitting next to each other, which is crucial for accurately calculating the probability.
###
BEST CHOICE: 2
RATIONALE: This approach breaks down the problem into manageable cases, allowing for a systematic count of the desired outcomes, making it more likely to yield an accurate solution.
###


In [22]:
res = None
votes = [
    choice for resp in vote_tokens if 
    (choice := parse_choice(workflow.tokenizer.decode(resp))) is not None
]

if len(votes) > 0:
    best = Counter(votes).most_common(1)[0][0]
    res, _ = workflow.step(
        [
            {
                'header': ('assistant', None),
                'parent_ids': [finish['id']] + [proposal_nodes[best-1]['id']]
            }
        ],
        compact=False,
        max_gen_len=256,
        temperature=0.7,
        top_p=0.9,
        seed=42,
        debug=False
    )

In [24]:
for vote in res:
    print(workflow.tokenizer.decode(vote))
    print('###')

ANSWER: To solve the problem, we count the total number of ways to choose 3 knights out of 25, which is $\binom{25}{3} = 2300$, and then count the number of ways to choose 3 knights where at least two of them are sitting next to each other. We find that there are 25 ways to choose two knights who are sitting next to each other and then choose the third knight from the remaining knights, and 25 ways to choose three knights who are all sitting next to each other. However, we have overcounted by 25, so the total number of ways to choose 3 knights where at least two of them are sitting next to each other is 50. Therefore, the probability $P$ is $\frac{50}{\binom{25}{3}} = \frac{50}{2300} = \frac{1}{46}$. The sum of the numerator and denominator is $1+46 = \boxed{47}$.
###
